In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.colors as colors
import folium
import pickle
import numpy as np
from ipywidgets import interact
import json

In [2]:
pickle_file="./Data/uni_df.pkl"
df=pickle.load(open(pickle_file,'rb'))

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.feature_selection import VarianceThreshold
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.feature_selection import RFE
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

def create_target_and_covariate_df(path_to_pkl):
    '''
    path_to_pkl: path to the pickle file.
    outputs two dataframes, one for the independant variables one for the dependant variables
    '''
    
    uni_df = pd.read_pickle(path_to_pkl)
    uni_df = uni_df.drop(columns=['Area', 'Year'])
    target_variables_df = uni_df[['(GDP, million $)', '(Consumer price indices, %)']]
    covariates_df = uni_df.drop(columns=['(GDP, million $)', '(Consumer price indices, %)'])
    
    return covariates_df, target_variables_df


def drop_feature_pearson_correlation(threshold, target_variable, target_variable_name, dataframe):
    
    '''
    threshold: the minimum amount of correlation required to keep the feature
    target_variable_name: string GDP or CPI
    normalised_dataset: the normalised dataset of feature
    target_variable: pandas series that contains the value of the target_varibale_name
    that we add to the normalised dataset
    
    '''
    copy_dataframe = dataframe.copy()
    copy_dataframe[target_variable_name] = target_variable
    cor = copy_dataframe.corr()
    cor_target = abs(cor[target_variable_name])
    
    relevant_features = cor_target[cor_target > threshold]
    
    return list(relevant_features.keys())

def drop_too_corelated_featues(threshold, dataframe):
    
    corr_matrix = dataframe.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    
    return dataframe.drop(dataframe[to_drop], axis=1)
    

def feature_augmentation(degree, covariates_df):
    poly = PolynomialFeatures(degree)
    output_nparray =  poly.fit_transform(covariates_df)

    
    output_df = pd.DataFrame(output_nparray, columns = poly.get_feature_names(covariates_df.columns))
    
    return output_df

def split_and_standardization_dataset(target_variables, covariates, test_size, random, type_return = 'numpy'  ):
    
    '''
    
    target_variables: pandas dataframe that contains the target variables
    covariates: pandas dataframe that contains the independant variables
    test_size: the proportion of the dataset to include in the test split
    type_return: 'numpy' if return numpy array, 'pandas' if return pandas dataframe
    '''
    target_variables_numpy = target_variables.to_numpy()
    covariates_numpy = covariates.to_numpy()
    X_train, X_test, Y_train, Y_test = train_test_split(covariates_numpy, target_variables_numpy, test_size=test_size, random_state = random)
    scaler = preprocessing.StandardScaler().fit(X_train)
    X_train_normalized = scaler.transform(X_train)
    X_test_normalized = scaler.transform(X_test)
    
    if type_return == 'numpy':
        
        return X_train_normalized, X_test_normalized, Y_train, Y_test
    
    elif type_return == 'pandas':
        
        X_test_normalized_df = pd.DataFrame(X_test_normalized, columns = list(covariates.columns))
        X_train_normalized_df = pd.DataFrame(X_train_normalized,columns= list(covariates.columns))
        Y_train_df = pd.DataFrame(Y_train, columns= list(target_variables.columns))
        Y_test_df = pd.DataFrame(Y_test, columns= list(target_variables.columns))
        
        return X_train_normalized_df, X_test_normalized_df, Y_train_df, Y_test_df

def fit_model_lasso(regularisation_parameters, covariates_df, target_df, nb_fold_CV):
    
    lasso = Lasso()
    
    parameters = {'alpha': regularisation_parameters}
    
    lasso_regressor = GridSearchCV(lasso, parameters, scoring = 'neg_mean_squared_error', cv = nb_fold_CV)
    lasso_regressor.fit(covariates_df, target_df)

    best_param = lasso_regressor.best_params_['alpha']
    print('The best regularization parameter is ', best_param)


    lasso = Lasso(alpha=best_param)
    lasso.fit(covariates_df, target_df)
    return lasso.coef_
    
    
    
def RFECV_lasso_2(covariate, target,  random, nb_fold = 5,):
    
    cols = list(covariate.columns)
    X_train_, X_test_, Y_train_, Y_test_ = split_and_standardization_dataset(target, covariate, 0.2, type_return='numpy', random = random)
    #print('shape of Y_train_', Y_train_.shape, 'type of Y_train_', type(Y_train_))
    model = Lasso()
    
    rfecv = RFECV(estimator = model, step = 1, cv = nb_fold, scoring = 'neg_mean_squared_error')
    rfecv.fit(X_train_, np.ravel(Y_train_))
    print("Optimal number of features : %d" % rfecv.n_features_)
    
    temp = pd.Series(rfecv.support_,index = cols)
    selected_features = temp[temp==True].index

    print(selected_features)
    

    # plt.figure()
    # plt.xlabel("Number of features selected")
    # plt.ylabel("Cross validation score")
    # plt.plot(range(1, len(rfecv.grid_scores_) + 1), rfecv.grid_scores_)
    # plt.show()
        
    return selected_features



    

def main():

    RANDOM_SEED = 29

    params = {

        'target' : '(GDP, million $)',
        'name of target': 'GDP',
        'pearson correlation threshold': 0.4,
        'inter correlation threshold': 0.9, 
        'nb_fold_CV': 5, 
        'degree augmentation': 1




    }

    covariates_df, target_variables_df = create_target_and_covariate_df('./Data/uni_df.pkl')
    target_variables_df.to_pickle('target.pkl')

    ### Below we are going to select the top 20 features in production:

    Production_cov_df = covariates_df.filter(regex= 'production|Production')
    summed_df = Production_cov_df.sum()
    keys = summed_df.keys()
    values = summed_df.values
    sorted_keys = [key for _,key in sorted(zip(values,keys))]
    Production_cov_df = Production_cov_df[sorted_keys[-20:]]
    selected_features_production = list(Production_cov_df.columns.values) # Selected features for top 20 prod features in volumne

    cropped_word_selected_prod = [" ".join(string.split()[:-3]) for string in selected_features_production] # Same as the list above with only the important words kept


    

   #-------------------------Below we are selecting the features in export that have been selected previously with the production--------------------------------------
    export_df = covariates_df.filter(regex= 'export')
    

    columns_to_keep_export = []

    for column_export in list(export_df.columns.values):

        for columns_prod in cropped_word_selected_prod:

            if columns_prod in column_export:

                columns_to_keep_export.append(column_export)


    #-------------------------Below we are selecting the features in import that have been selected previously with the production--------------------------------------
    import_df = covariates_df.filter(regex= 'import')
    

    columns_to_keep_import = []

    for column_import in list(import_df.columns.values):

        for columns_prod in cropped_word_selected_prod:

            if columns_prod in column_import:

                columns_to_keep_import.append(column_import)

    


    final_features_kept = selected_features_production + columns_to_keep_export + columns_to_keep_import  # All the selected features
    print('final features', final_features_kept)

    # summed_covariates_df = covariates_df.sum()
    # keys = summed_covariates_df.keys()
    # values = summed_covariates_df.values
    
    # sorted_keys = [key for _,key in sorted(zip(values,keys))]

    # covariates_df = covariates_df[sorted_keys[-30:]]
    covariates_df = covariates_df[final_features_kept]
    
    # covariates_df = feature_augmentation(2, covariates_df)
    print('list of all features', list(covariates_df.columns.values))
    list_selected_features_GDP = drop_feature_pearson_correlation(params['pearson correlation threshold'], target_variables_df[params['target']], params['name of target'], covariates_df)
    print('amount of selected features', len(list_selected_features_GDP))
    print('selected features', list_selected_features_GDP)
    covariate_reduced_df = covariates_df[list_selected_features_GDP[:-1]]

    covariate_reduced_df = drop_too_corelated_featues(params['inter correlation threshold'], covariate_reduced_df)
    covariate_reduced_df.to_pickle("reduced_df_2.pkl")
    print('list of selected features after reduction', list(covariate_reduced_df.columns.values))
    # covariate_reduced_df = feature_augmentation(params['degree augmentation'], covariate_reduced_df)

    # selected_features = RFECV_lasso_2(covariate_reduced_df, target_variables_df[[params['target']]], random = RANDOM_SEED)
    # selected_covariate = covariate_reduced_df[selected_features]

    regularisation_parameters = np.linspace(start = 0.01, stop= 1, num = 20)

    # covariate_reduced_df = covariate_reduced_df[list(selected_covariate.columns.values)]

    target_df = target_variables_df[params['target']]

    nb_fold_CV = params['nb_fold_CV']

    param_lasso = fit_model_lasso(regularisation_parameters, covariate_reduced_df, target_df, nb_fold_CV = nb_fold_CV )

    keys = list(covariate_reduced_df.columns.values)
    #keys = selected_features
    values = param_lasso
   
    return dict(zip(keys, values))

weights=main()

final features ['Sweet potatoes Crops Production tonnes', 'Soybeans Crops Production tonnes', 'Tomatoes Crops Production tonnes', 'Oats Crops Production tonnes', 'Buffaloes Livestock production Head', 'Cassava Crops Production tonnes', 'Barley Crops Production tonnes', 'Rabbits and hares Livestock production Head', 'Maize Crops Production tonnes', 'Goats Livestock production Head', 'Turkeys Livestock production Head', 'Ducks Livestock production Head', 'Sugar beet Crops Production tonnes', 'Potatoes Crops Production tonnes', 'Wheat Crops Production tonnes', 'Pigs Livestock production Head', 'Cattle Livestock production Head', 'Sheep Livestock production Head', 'Sheep and Goats Livestock production Head', 'Chickens Livestock production Head', 'Barley Food export quantities tonnes', 'Buffaloes Live animals export quantities Head', 'Cassava Food export quantities tonnes', 'Cattle Live animals export quantities Head', 'Chickens Live animals export quantities Head', 'Ducks Live animals expo

//anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 88269505930363.5, tolerance: 616213877638.2041
  positive)
//anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 52106333384857.0, tolerance: 641076863381.032
  positive)


The best regularization parameter is  0.01


//anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25251481765867.0, tolerance: 650584559072.4933
  positive)
//anaconda3/envs/ada/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25251481765867.0, tolerance: 650584559072.4933
  positive)


In [4]:
import matplotlib.colors as colors

def visualise_world_data_folium(df, to_visualise, year, log=True,log2=False):
    
    if log2:
        log=False
    if log:
        log2=False
        
    # Defining color palette
    color_scale = sns.cubehelix_palette(9)
    
    # importing geojson and transforming to pandas
    geo_data=json.load(open("./Data/world-countries.json"))
    dics=geo_data['features']
    clean_dics=[]
    for country in dics:
        clean_dics.append({'Country':country['properties']['name'],
                          'geometry':country['geometry']})
    geo_df=pd.DataFrame(clean_dics)
    
    # cropping to df to data of interest
    df_visu=df[df.Year==year][['Area',to_visualise]]

    # Merging with geo data
    df_visu=geo_df.merge(df_visu,how='left',left_on='Country',right_on='Area')
    df_visu=df_visu.dropna()
    
    if log:
        df_visu['to_plot']=df_visu[to_visualise].apply(lambda x : np.log10(x))
        
    def log2_scale(x):
        out=np.sign(x)*np.log10(1+np.abs(x))
        return out
        
    if log2:
        df_visu['to_plot']=df_visu[to_visualise].apply(log2_scale)
    
    # creating bins for color scaling
    ma_value=df_visu['to_plot'].max()
    mi_value=df_visu['to_plot'].min()
    bins=np.linspace(mi_value,ma_value,8)
    
    # creating Json string for folium
    features=[]
    for _,row in df_visu.iterrows():
        color=np.digitize(row['to_plot'],bins)
        val=row[to_visualise]
        feature={
            'type' : 'Feature',
            
            'properties':{'name':row['Country'],
                          'value': '{:.2E}'.format(val),
                          'color':colors.to_hex(color_scale[color])},
            'geometry':row['geometry']
            }
        features.append(feature)
    
    def style(feature):
        
        if feature['properties']['value']==np.nan:
            print("lol")
            opac=0
        else:
            opac=0.8
        return {'fillOpacity':opac,
                   'weight':0.1,
                   'fillColor':feature['properties']['color']}
    geo_data=folium.GeoJson({'type':'FeatureCollection','features':features},
                            style_function=style,
                            tooltip=folium.features.GeoJsonTooltip(['name','value']))
    m=folium.Map()
    geo_data.add_to(m)
    return m

In [5]:
weights

{'Soybeans Crops Production tonnes': 0.016904551719883004,
 'Tomatoes Crops Production tonnes': 0.04600299893471108,
 'Maize Crops Production tonnes': 0.019112555979080598,
 'Turkeys Livestock production Head': 0.00045609049200977524,
 'Maize Food export quantities tonnes': -0.06316454222878906,
 'Maize, green Food export quantities tonnes': 52.082838493928676,
 'Wheat Food export quantities tonnes': 0.0075799594660751575,
 'Cattle Live animals import quantities Head': 0.03663454785851955,
 'Oats Food import quantities tonnes': 0.48376570716305617,
 'Pigs Live animals import quantities Head': 0.05776106945901765,
 'Tomatoes Food import quantities tonnes': 2.5686557773141767,
 'Turkeys Live animals import quantities Head': 0.04929538371588062}

In [235]:
columns=list(weights.keys())
df=pickle.load(open(pickle_file,'rb'))
df=df.set_index(['Area','Year'])
prod_to_plot=pd.DataFrame(index=df.index)

In [248]:
dic_to_plot={}
for c in columns:
    print(c)
    if 'Production' in c or 'production' in c:
        if len(df.filter(regex=c).columns)==0:
            print('{} not found'.format(c))
        else:
            dic_to_plot.update(df.filter(regex=c).to_dict())
    else:
        s=re.split(' Food|Live ',c)[0]
        s='^'+s
        s=(s+'.*Production.*tonnes$|'
           +s+'.*Production.*Head$|'
           +s+'.*production.*tonnes$|'
           +s+'.*production.*Head')
        if len(df.filter(regex=s).columns)==0:
            print('{} not found'.format(c))
        else:
            dic_to_plot.update(df.filter(regex=s).to_dict())
prod_to_plot=pd.DataFrame(dic_to_plot)

Soybeans Crops Production tonnes
Tomatoes Crops Production tonnes
Maize Crops Production tonnes
Turkeys Livestock production Head
Maize Food export quantities tonnes
Maize, green Food export quantities tonnes
Wheat Food export quantities tonnes
Cattle Live animals import quantities Head
Oats Food import quantities tonnes
Pigs Live animals import quantities Head
Tomatoes Food import quantities tonnes
Turkeys Live animals import quantities Head


In [249]:
prod_to_plot.head(1)

,,Soybeans Crops Production tonnes,Tomatoes Crops Production tonnes,Maize Crops Production tonnes,Turkeys Livestock production Head,"Maize, green Crops Production tonnes",Wheat Crops Production tonnes,Cattle Livestock production Head,Oats Crops Production tonnes,Pigs Livestock production Head
Afghanistan,1970,0.0,0.0,667000.0,0.0,0.0,2081000.0,3700000.0,0.0,0.0


In [250]:
prod_to_plot=prod_to_plot.reset_index().rename(columns={'level_0':'Area','level_1':'Year'})

In [251]:
import os
for c in prod_to_plot.columns:
    if c!='Year' and c!='Area':
        os.mkdir('./Data/ResultsJulien/Producers/{}'.format(c))
        for year in range(1970,2015,1):
            m=visualise_world_data_folium(prod_to_plot,c,year,log2=True)
            save_name='./Data/ResultsJulien/Producers/{}/{}_{}.html'.format(c,c,year)
            m.save(save_name)

In [6]:
def compute_self_suficiency(df,weights=None):
    
    # From the unified dataframe df, compute the self sufficiency score for each year for each country
    # if a paramter of weights is given as a dict, the method returns the aggregated score.
    
    
    #Useful method to manipulate names
    def drop_words( s , w=1 , end=True):
        if end:
            return s.rsplit(' ',w)[0]
        else:
            return s.split(' ',w)[-1]
    
    df=df.set_index(['Area','Year'])
    
    #Getting the columns corresponding to import, export and production
    import_cols=[col for col in df.columns if 'import' in col.lower()]
    export_cols=[col for col in df.columns if 'export' in col.lower()]
    prod_cols=[col for col in df.columns if 'production' in col.lower()]
    
    #Initializing new dataframe
    scores=pd.DataFrame(index=df.index)
    #Dict with columns names
    columns={}
    
    #Generating scores
    for i,col in enumerate(import_cols):
        key=col+prod_cols[i]+export_cols[i]
        scores[key]=(df[prod_cols[i]]*100/(
                                    df[prod_cols[i]]+df[import_cols[i]]-df[export_cols[i]]))
        columns[key]=drop_words(col,3)
        
    #If no weights, return scores without aggregate
    if weights==None:
        scores=scores.rename(columns=columns)
        return scores
    
    features=[w for w in weights.keys()]
    temp=pd.DataFrame(index=df.index)
    
    #replacing na with 0 to avoid na aggregated scores
    scores=scores.fillna(0)
    
    #Selecting features of interest
    temp_dic={}
    for feat in features:
        temp_dic.update(scores.filter(regex=feat))
    temp=pd.DataFrame(temp_dic)
    temp.columns=features
    
    #Aggregating the scores
    temp['Agg']=0
    for feat in features:
        temp['Agg']=temp['Agg']+weights[feat]*temp[]
    scores=temp['Agg']
    
    return scores

In [7]:
df=pickle.load(open(pickle_file,'rb'))

In [8]:
scores=compute_self_suficiency(df,weights)

ValueError: Length mismatch: Expected axis has 9 elements, new values have 12 elements